## Text Classification Evaluation - Entailment v/s Contradiction

This sample shows how use the evaluate a group of models against a given set of metrics for the `text-classification` task. 

### Evaluation dataset
The Multi-Genre Natural Language Inference Corpus, or MNLI is a crowd sourced collection of sentence pairs with textual entailment annotations. Given a premise sentence and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The [MNLI](https://huggingface.co/datasets/glue) dataset is a subset of the larger [General Language Understanding Evaluation](https://gluebenchmark.com/) dataset. A copy of this dataset is available in the [glue-mnli](./glue-mnli/) folder. 

### Model
The goal of evaluating models is to compare their performance on a variety of metrics. `text-classification` is generic task type that can be used for scenarios such as sentiment analysis, emotion detection, grammar checking, spam filtering, etc. As such, the models you pick to compare must be finetuned for same scenario. Given that we have the entailment v/s contradiction dataset, we would like to look for models finetuned for this specific scenario. We will compare `roberta-large-mnli`, `microsoft-deberta-large-mnli` and `microsoft-deberta-base-mnli` in this sample, which are available in the `azureml` system registry.

### Outline
* Setup pre-requisites such as compute.
* Pick the models to evaluate.
* Pick and explore evaluate data.
* Configure the evaluation jobs.
* Run the evaluation jobs.
* Review the evaluation metrics. 

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name
* Check or create compute. A single GPU node can have multiple GPU cards. For example, in one node of `Standard_ND40rs_v2` there are 8 NVIDIA V100 GPUs while in `Standard_NC12s_v3`, there are 2 NVIDIA V100 GPUs. Refer to the [docs](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) for this information. The number of GPU cards per node is set in the param `gpus_per_node` below. Setting this value correctly will ensure utilization of all GPUs in the node. The recommended GPU compute SKUs can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ncv3-series) and [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series).

Install dependencies by running below cell. This is not an optional step if running in a new environment.

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity
%pip install datasets==2.9.0

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "<SUBSCRIPTION_ID>",
        resource_group_name =  "<RESOURCE_GROUP>",
        workspace_name =  "<WORKSPACE_NAME>"
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

experiment_name = "mnli-evaluation"

# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'gpu-cluster-big'
compute_cluster = "gpu-cluster-big"
try:
    compute = workspace_ml_client.compute.get(compute_cluster)
except Exception as ex:
    compute = AmlCompute(
        name = compute_cluster, 
        size= "Standard_ND40rs_v2",
        max_instances= 2 # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time())) 


Below snippet will allow us to query number of GPU's present on the compute. We can use it to set `gpu_per_node` to ensure utilization of all GPUs in the node.

In [ ]:
# This is the number of GPUs in a single node of the selected 'vm_size' compute. 
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 1 # default value
gpu_count_found = False
ws_computes = workspace_ml_client.compute.list_sizes()
for ws_compute in ws_computes:
    if ws_compute.name == compute.size:
        gpu_count_found = True
        gpus_per_node = ws_compute.gpus
        print(f"Number of GPU's in compute {ws_compute.name} are {ws_compute.gpus}")
# if gpu_count_found not found, then print an error 
gpu_count_found or print(f"Error: Number of GPU's in compute {compute.size} not found")

### 2. Pick the models to evaluate

Verify that the models selected for evaluation are available in system registry

In [ ]:
# need to specify model versions until the bug to support fetching the latest version using latest label is fixed
models = [ {'name': 'roberta-large-mnli', 'version': '2'},
          {'name': 'microsoft-deberta-large-mnli', 'version': '2'},
          {'name': 'microsoft-deberta-base-mnli', 'version': '2'}]
for model in models:
    model = registry_ml_client.models.get(model['name'], version=model['version'])
    print (model.id)

### 3. Pick the test dataset for evaluation
A copy of the MNLI is available in the [ glue-mnli](./glue-mnli/) folder. The next few cells show basic data preparation:
* Visualize some data rows
* Replace numerical categories in data with the actual string labels. This mapping is available in the [./glue-mnli/label.json](./glue-mnli/label.json). This step is needed because the selected models will return labels such `CONTRADICTION`, `CONTRADICTION`, etc. when running prediction. If the labels in your ground truth data are left as `0`, `1`, `2`, etc., then they would not match with prediction labels returned by the models.
* We want this sample to run quickly, so save smaller dataset containing 10% of the original. 

In [ ]:
import os
dataset_dir = "./glue-mnli"
data_file="train.jsonl"

# load the train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd
pd.set_option('display.max_colwidth', 0) # set the max column width to 0 to display the full text
df = pd.read_json(os.path.join(dataset_dir,data_file), lines=True)
df.head()

In [ ]:
# load the id2label json element of the label.json file into pandas table with keys as 'label' column of int64 type and values as 'label_string' column as string type
import json
label_file="label.json"
with open(os.path.join(dataset_dir,label_file)) as f:
    id2label = json.load(f)
    id2label = id2label['id2label']
    label_df = pd.DataFrame.from_dict(id2label, orient='index', columns=['label_string'])
    label_df['label'] = label_df.index.astype('int64')
    label_df = label_df[['label', 'label_string']]
label_df.head()

In [ ]:
# join the train, validation and test dataframes with the id2label dataframe to get the label_string column
df =df.merge(label_df, on='label', how='left')
# concat the premise and hypothesis columns to with "[CLS]" in the beginning and "[SEP]" in the middle and end to get the text column
df['text'] = "[CLS] " + df['premise'] + " [SEP] " + df['hypothesis'] + " [SEP]"
# drop the idx, premise and hypothesis columns as they are not needed
df = df.drop(columns=['idx', 'premise', 'hypothesis'])
df.head()

In [ ]:
# save 10% of the rows from the train, validation and test dataframes into files with small_ prefix in the ./dataset_dir folder
small_data_file = "small_train.jsonl"
df.sample(frac=0.1).to_json(os.path.join(dataset_dir,small_data_file), orient='records', lines=True)

### 4. Submit the evaluation jobs using the the model and data as inputs
 
Create the job that uses the `model_evaluation_pipeline` component. We will submit one job per model. 

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import CommandComponent, PipelineComponent, Job, Component
from azure.ai.ml import PyTorchDistribution, Input
from azure.ai.ml.constants import AssetTypes

# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(name="model_evaluation_pipeline", label="latest")

# define the pipeline job
@pipeline()
def evaluation_pipeline(
    mlflow_model
):
    evaluation_job = pipeline_component_func( 
        
        # specify the foundation model available in the azureml system registry or a model from the workspace
       # mlflow_model = Input(type=AssetTypes.MLFLOW_MODEL, path=f"{mlflow_model_path}"),
        mlflow_model = mlflow_model,
       
        # test data
        test_data = Input(type="uri_file", path=os.path.join(dataset_dir, small_data_file)),
        # The following parameters map to the dataset fields
        input_column_names = "text", 
        label_column_name = "label_string", 
        # Evaluation settings
        task = "text-classification",
        # config file containing the details of evaluation metrics to calculate
        evaluation_config = Input(type="uri_file", path="./eval_config.json"),
    )
    return {
        "evaluation_result": evaluation_job.outputs.evaluation_result
    }

Submit the jobs, passing the model as a parameter to the pipeline created in the above step.

In [ ]:
# submit the pipeline job for each model that we want to evaluate
# you could consider submitting the pipeline jobs in parallel, provided your cluster has multiple nodes
pipeline_jobs = []

for model in models:
    model_object = registry_ml_client.models.get(model['name'], version=model['version'])
    pipeline_object = evaluation_pipeline(mlflow_model=Input(type=AssetTypes.MLFLOW_MODEL, path=f"{model_object.id}"),)
    # don't reuse cached results from previous jobs
    pipeline_object.settings.force_rerun = True
    pipeline_object.settings.default_compute = compute_cluster
    pipeline_job = workspace_ml_client.jobs.create_or_update(pipeline_object, experiment_name=experiment_name)
    # add model['name'] and pipeline_job.name as key value pairs to a dictionary
    pipeline_jobs.append({'model_name': model['name'], 'job_name': pipeline_job.name})
    # wait for the pipeline job to complete
    workspace_ml_client.jobs.stream(pipeline_job.name)

### 5. Review evaluation metrics
Viewing the job in AzureML studio is the best way to analyze logs, metrics and outputs of jobs. You can create custom charts and compare metics across different jobs. See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics?tabs=interactive#view-jobsruns-information-in-the-studio to learn more. 

However, we may need to access and review metrics programmatically for which we will use MLflow, which is the recommended client for logging and querying metrics.

In [ ]:
import mlflow, json
mlflow_tracking_uri = workspace_ml_client.workspaces.get(workspace_ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(mlflow_tracking_uri)

metrics_df = pd.DataFrame()
for job in pipeline_jobs:
    # concat 'tags.mlflow.rootRunId=' and pipeline_job.name in single quotes as filter variable
    filter = "tags.mlflow.rootRunId='" + job['job_name'] + "'"
    runs = mlflow.search_runs(experiment_names=[ experiment_name ], filter_string = filter, output_format="list")
    # get the compute_metrics runs.
    # using a hacky way till 'Bug 2320997: not able to show eval metrics in FT notebooks - mlflow client now showing display names' is fixed
    for run in runs:
        # else, check if run.data.metrics.accuracy exists
        if 'accuracy' in run.data.metrics:
            # get the metrics from the mlflow run
            run_metric = run.data.metrics
            # add the model name to the run_metric dictionary
            run_metric['model_name'] = job['model_name']
            # convert the run_metric dictionary to a pandas dataframe
            temp_df = pd.DataFrame(run_metric,index=[0])
            # concat the temp_df to the metrics_df
            metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)

# move the model_name columns to the first column
cols = metrics_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_df = metrics_df[cols]
metrics_df.head()
            